<a href="https://colab.research.google.com/github/vitaliy-sharandin/data_science_projects/blob/master/portfolio/regression/Energy_price_demand_and_potential_supply.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Energy demand prediction of Spain's major cities

This project uses energy demand information from major Spanish cities to predict cumulative energy consumption.

# Dataset
Spanish major cities energy consumption and weather dataset
* https://www.kaggle.com/datasets/nicholasjhana/energy-consumption-generation-prices-and-weather



In [ ]:
!pip install -U -q datasets
!pip install -U -q ydata-profiling
!pip install -U -q feature_engine
!pip install -U -q Boruta
!pip install -U -q optuna
!pip install -U -q eli5
!pip install statsforecast

# EDA


In [ ]:
from datasets import load_dataset
from ydata_profiling import ProfileReport
import pandas as pd
from feature_engine.encoding import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from boruta import BorutaPy
import xgboost as xgb
import optuna
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
energy_consumption_dataset = load_dataset("vitaliy-sharandin/energy-consumption-hourly-spain")
energy_consumption_weather_dataset = load_dataset("vitaliy-sharandin/energy-consumption-weather-hourly-spain")
energy_df = energy_consumption_dataset['train'].to_pandas()
weather_df = energy_consumption_weather_dataset['train'].to_pandas()

## Energy dataset

In [ ]:
energy_df['time'] = pd.to_datetime(energy_df['time'], errors="coerce", utc=True)
energy_df = energy_df.set_index('time')
energy_df.index = pd.to_datetime(energy_df.index,utc=True)
energy_df=energy_df.asfreq('h')

In [ ]:
# profile = ProfileReport(energy_df, title="Energy data report", dark_mode=True)
# profile.to_notebook_iframe()

In [ ]:
energy_df.drop(['generation hydro pumped storage aggregated','forecast wind offshore eday ahead',
                'generation marine','generation fossil coal-derived gas','generation fossil oil shale',
                'generation fossil peat','generation geothermal','generation marine','generation wind offshore'], axis=1, inplace=True)

In [ ]:
def categorize_variables(target, df_train, cat_numeric_unique_threshold=10):
  target = target
  categorical_numeric = [var for var in df_train.columns if df_train[var].dtype!='O' and var!=target and df_train[var].nunique()<=cat_numeric_unique_threshold]
  continuous = [var for var in df_train.columns if df_train[var].dtype!='O' and var!=target and var not in categorical_numeric]
  mixed = [var for var in df_train.columns if pd.api.types.infer_dtype(df_train[var]) == 'mixed']
  categorical_object = [var for var in df_train.columns if df_train[var].dtype=='O' and var not in mixed]
  sorted_features = [target]+categorical_numeric+continuous+categorical_object+mixed
  print('Total columns: '+str(df_train.columns.size)+'\nColumns after sorting: '+str(len(sorted_features)))
  return target, categorical_numeric, continuous, mixed, categorical_object
target, categorical_numeric, continuous, mixed, categorical_object = categorize_variables('total load actual', energy_df)

In [ ]:
energy_df[categorical_numeric+continuous]=energy_df[categorical_numeric+continuous].fillna(energy_df[categorical_numeric+continuous].mean())
energy_df[target] = energy_df[target].interpolate(method='linear')

**Insights**
* Some features were highly correlated, but we'll deal with them in feature selection phase.
* 2 features were completely missing and 6 were constant, so we dropped them.
* Categorical variables were filled with mean values.
* Target variable had several missing values as well, so we used linear interpolation to fill them.

## Target variable analysis

In [ ]:
target_decompose = seasonal_decompose(energy_df[:168][target], model='additive')
plot = target_decompose.plot()
plot.set_size_inches((16, 9))
plt.show()

In [ ]:
plot_acf(energy_df[target], lags=5)
plt.show()
plot_pacf(energy_df[target], lags=5)
plt.show()

**Insights**
* Series are non-stationary.
* Target variable has daily seasonality.
* The dataset has properties of AR process what stems from fast cutoff in PACF chart, this will become handy when we test ARIMA based models.

## Weather dataset EDA

In [ ]:
weather_df['dt_iso'] = pd.to_datetime(weather_df['dt_iso'], utc=True)

In [ ]:
# profile = ProfileReport(weather_df, title="Weather data report")
# profile.to_notebook_iframe()

In [ ]:
weather_df.drop(['rain_3h','weather_main','weather_icon'], axis=1, inplace=True)
weather_df = weather_df[weather_df['dt_iso'].isin(energy_df.index) & (~weather_df.duplicated(['dt_iso','city_name']))]

In [ ]:
weather_df = weather_df.set_index(['dt_iso','city_name'])
weather_df = weather_df.unstack('city_name')
weather_df.columns = ['_'.join(col).strip() for col in weather_df.columns.values]

#### Merging energy and weather dataset

In [ ]:
energy_weather_df = energy_df.join(weather_df, how='inner')

# Feature selection and engineering

## Transforming variables

In [ ]:
target, categorical_numeric, continuous, mixed, categorical_object = categorize_variables('total load actual', energy_weather_df)

train = energy_weather_df[:-168]
test = energy_weather_df[-168:]

encoder = OrdinalEncoder(
    variables=categorical_object,
    encoding_method='ordered'
)

train = encoder.fit_transform(train, train[target])
test = encoder.fit_transform(test, test[target])

## Feature selection

In [ ]:
from xgboost import XGBRegressor
import eli5

# params = {
#     'objective': 'reg:squarederror',
#     'random_state': 42,
#     'n_jobs': -1,
#     'learning_rate': 0.1,
#     'max_depth': 3,
#     'min_child_weight': 1,
#     'gamma': 0,
#     'subsample': 0.8,
#     'colsample_bytree': 0.8,
#     'reg_alpha': 0,
#     'reg_lambda': 1
# }

xgb = XGBRegressor()
xgb.fit(X_train, y_train)

display(eli5.show_weights(xgb, feature_names = X_test.columns.tolist()))

In [ ]:
# %%time
# lgb_regressor = lgb.LGBMRegressor()

# feat_selector = BorutaPy(lgb_regressor, n_estimators='auto', random_state=42)
# feat_selector.fit(X_train.values, y_train.values)
# selected_rf_features = pd.DataFrame({'Feature':list(X_train.columns),
#                                        'Ranking':feat_selector.ranking_}).sort_values(by='Ranking')
# selected_rf_features.nsmallest(40, 'Ranking').plot.barh(x='Feature',figsize=(24,5))

In [ ]:
# black_list_features = ['generation biomass', 'generation fossil brown coal/lignite',
#        'generation fossil gas', 'generation fossil hard coal',
#        'generation fossil oil', 'generation hydro pumped storage consumption',
#        'generation hydro run-of-river and poundage',
#        'generation hydro water reservoir', 'generation nuclear',
#        'generation other', 'generation other renewable', 'generation solar',
#        'generation waste', 'generation wind onshore','forecast solar day ahead',
#        'forecast wind onshore day ahead', 'price actual']

# energy_weather_df = energy_weather_df[energy_weather_df.columns.difference(black_list_features)]

# Model selection

## ARIMA and others with statsforecast model test

First, we transform data into statsforecast format and also order columns for statsforecast framework so that it can use

In [ ]:
def transform_to_statsforecast_format(df):
  energy_weather_df_forecast = df.copy()
  energy_weather_df_forecast['unique_id'] = 'Energy_weather_Spain'
  energy_weather_df_forecast['ds'] = energy_weather_df_forecast.index.tz_localize(None)
  energy_weather_df_forecast = energy_weather_df_forecast.rename(columns={'total load actual':'y'})

  exogenous_variables = energy_weather_df_forecast.columns.difference(['unique_id', 'ds', 'y']).tolist()
  cols = ['unique_id', 'ds', 'y'] + exogenous_variables
  return energy_weather_df_forecast.reindex(columns=cols)
train = transform_to_statsforecast_format(train)
test = transform_to_statsforecast_format(test)

First, let's see how ARIMA, ETS, Theta and CES models behave.

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

season_length = 24
prediction_horizon = 168


models = [
    AutoARIMA(season_length=season_length)
]

sf = StatsForecast(
    models=models,
    freq='H',
    n_jobs=-1
)

fcst = sf.forecast(df=train, h=prediction_horizon, X_df=test[test.columns.difference(['y'])], level=[95])
fcst = fcst.reset_index()
display(fcst.head())
StatsForecast.plot(test['y'], fcst, max_insample_length=28*2, engine='plotly')

In [ ]:
# !pip install pmdarima
# import pmdarima as pm
# SARIMAX_model = pm.auto_arima(y_train[-8760:], X=X_train[-8760:],
#                            start_p=1, start_q=0,
#                            start_P=1, start_Q=0,
#                            test='adf',
#                            max_p=2, max_q=0,
#                            max_P=2, max_Q=0,
#                            m=24,
#                            seasonal=True,
#                            stepwise=True,
#                            trace=True,
#                            maxiter=10)

# def sarimax_forecast(SARIMAX_model, periods):
#     # Forecast
#     n_periods = periods

#     predicted, confint = SARIMAX_model.predict(n_periods=n_periods,
#                                             return_conf_int=True)
#     index_of_fc = predicted.index

#     # make series for plotting purpose
#     predicted_series = pd.Series(predicted, index=index_of_fc)
#     lower_series = pd.Series(confint[:, 0], index=index_of_fc)
#     upper_series = pd.Series(confint[:, 1], index=index_of_fc)

#     # Plot
#     plt.figure(figsize=(15,7))
#     plt.plot(y_test, color='#1f76b4')
#     plt.plot(predicted_series, color='darkgreen')
#     plt.fill_between(lower_series.index,
#                     lower_series,
#                     upper_series,
#                     color='k', alpha=.15)

#     plt.title("SARIMAX Forecast")
#     plt.show()

# sarimax_forecast(SARIMAX_model, periods=168)

As we can see, SARIMAX model did not perform well

## Tree and Deep learning model tests with MLForecast framework

In [ ]:
!pip install -U -q mlforecast
!pip install -U -q neuralforecast
!pip install -U -q datasetsforecast
!pip install -U -q pmdarima

In [ ]:
from mlforecast import MLForecast
from datasetsforecast.losses import rmse
import pmdarima as pm
from sklearn.preprocessing import MinMaxScaler
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM
from neuralforecast.losses.pytorch import DistributionLoss

Preparing dataset in MLForecast format

In [ ]:
energy_weather_df_mlforecast = energy_weather_df.copy()
energy_weather_df_mlforecast['unique_id'] = 'Energy_weather_Spain'
energy_weather_df_mlforecast['ds'] = energy_weather_df_mlforecast.index.tz_localize(None)
energy_weather_df_mlforecast = energy_weather_df_mlforecast.rename(columns={'total load actual':'y'})

In [ ]:
train = energy_weather_df_mlforecast[:-168]
test = energy_weather_df_mlforecast[-168:]

encoder = OrdinalEncoder(
    variables=categorical_object,
    encoding_method='ordered'
)

train = encoder.fit_transform(train, train['y'])
test = encoder.fit_transform(test, test['y'])

Tree model hyperparameters search

In [ ]:
# %%time
# warnings.filterwarnings('ignore')

# def objective(trial):

#     model_name = trial.suggest_categorical("classifier", ['LGBMRegressor', 'XGBRegressor'])

#     if model_name == "LGBMRegressor":
#       params = {
#         "objective": "regression",
#         "metric": "rmse",
#         "n_estimators": 1000,
#         "verbosity": -1,
#         "bagging_freq": 1,
#         "learning_rate": trial.suggest_float("learning_rate_light", 1e-3, 0.1, log=True),
#         "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
#         "subsample": trial.suggest_float("subsample_light", 0.05, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree_light", 0.05, 1.0),
#         "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
#       }
#       model = lgb.LGBMRegressor(**params)

#     elif model_name == "XGBRegressor":
#       params = {
#       'max_depth': trial.suggest_int('max_depth', 1, 10),
#       'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
#       'n_estimators': trial.suggest_int('n_estimators', 50, 500),
#       'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#       'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
#       'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
#       'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
#       'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
#       'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
#       'eval_metric': 'rmse',
#       'use_label_encoder': False
#       }
#       model = xgb.XGBRegressor(**params)

#     ml_forecast = MLForecast(models=[model],
#                    freq='H',
#                    lags=[1,2,12,24,168],
#                    date_features=['hour','day','week', 'month'],
#                    num_threads=6)

#     crossvalidation_df = ml_forecast.cross_validation(
#                                                       data=train,
#                                                       window_size=168,
#                                                       n_windows=3,
#                                                     )
#     rmse = crossvalidation_df.groupby(['unique_id', 'cutoff']).apply(lambda df: rmse(df['y'], df[model_name])).mean()

#     return rmse

# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=50, show_progress_bar=True)

# display(study.best_params)
# display(study.best_value)

Testing tree models

In [ ]:
# ml_forecast = MLForecast(models=[model],
#                    freq='H',
#                    lags=[1,2,3,12,24],
#                    date_features=['hour','day','week'],
#                    num_threads=6)

# ml_forecast.fit(train, target_col='y')
# y_pred = ml_forecast.predict(len(test.index))

# for model_name in ml_forecast.models:
#   display(mean_squared_error(test['y'], y_pred[model_name], squared=False))

#   pd.Series(ml_forecast.models_[model_name].feature_importances_, index=ml_forecast.ts.features_order_).sort_values(ascending=False).plot.bar(figsize=(32, 6),title=f'{model_name} feature importance')
#   plt.show()

#   plt.figure(figsize=(30, 6))
#   sns.lineplot(x=test.index, y=test['y'], label='Real')
#   sns.lineplot(x=test.index, y=y_pred[model_name], label='Predicted')
#   plt.title('Real vs Predicted Values')
#   plt.legend()
#   plt.show()

DL model hyperparameter search

In [ ]:
# %%time
# warnings.filterwarnings('ignore')

# def objective(trial):

#     model_name = trial.suggest_categorical("classifier", ['LGBMRegressor', 'XGBRegressor'])

#     if model_name == "LGBMRegressor":
#       params = {
#         "objective": "regression",
#         "metric": "rmse",
#         "n_estimators": 1000,
#         "verbosity": -1,
#         "bagging_freq": 1,
#         "learning_rate": trial.suggest_float("learning_rate_light", 1e-3, 0.1, log=True),
#         "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
#         "subsample": trial.suggest_float("subsample_light", 0.05, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree_light", 0.05, 1.0),
#         "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
#       }
#       model = lgb.LGBMRegressor(**params)

#     elif model_name == "XGBRegressor":
#       params = {
#       'max_depth': trial.suggest_int('max_depth', 1, 10),
#       'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
#       'n_estimators': trial.suggest_int('n_estimators', 50, 500),
#       'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#       'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
#       'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
#       'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
#       'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
#       'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
#       'eval_metric': 'rmse',
#       'use_label_encoder': False
#       }
#       model = xgb.XGBRegressor(**params)

#     ml_forecast = MLForecast(models=[model],
#                    freq='H',
#                    lags=[1,2,12,24,168],
#                    date_features=['hour','day','week', 'month'],
#                    num_threads=6)

#     crossvalidation_df = ml_forecast.cross_validation(
#                                                       data=train,
#                                                       window_size=168,
#                                                       n_windows=3,
#                                                     )
#     rmse = crossvalidation_df.groupby(['unique_id', 'cutoff']).apply(lambda df: rmse(df['y'], df[model_name])).mean()

#     return rmse

# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=50, show_progress_bar=True)

# display(study.best_params)
# display(study.best_value)

DL model testing

In [ ]:
import os, torch, gc
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"
torch.cuda.empty_cache()
gc.collect()

In [ ]:
prediction_horizon = 168

models = [LSTM(h=prediction_horizon,
               loss=DistributionLoss(distribution='Normal', level=[90]),
               max_steps=50,
               encoder_n_layers=2,
               encoder_hidden_size=200,
               context_size=8760,
               decoder_hidden_size=200,
               decoder_layers=2,
               learning_rate=1e-3,
               scaler_type='standard',
               futr_exog_list=['onpromotion'])]

neural_forecast = NeuralForecast(models=models, freq='H')
neural_forecast.fit(train)

y_pred = neural_forecast.predict(futr_df=test)

for model_name in neural_forecast.models:
  display(mean_squared_error(test['y'], y_pred[model_name], squared=False))

  pd.Series(neural_forecast.models_[model_name].feature_importances_, index=neural_forecast.ts.features_order_).sort_values(ascending=False).plot.bar(figsize=(32, 6),title=f'{model_name} feature importance')
  plt.show()

  plt.figure(figsize=(30, 6))
  sns.lineplot(x=test.index, y=test['y'], label='Real')
  sns.lineplot(x=test.index, y=y_pred[model_name], label='Predicted')
  plt.title('Real vs Predicted Values')
  plt.legend()
  plt.show()

In [ ]:
y_pred =  neural_forecast.predict(futr_df=test)

for model_name in neural_forecast.models:
  display(mean_squared_error(test['y'], y_pred[model_name], squared=False))

  pd.Series(neural_forecast.models_[model_name].feature_importances_, index=neural_forecast.ts.features_order_).sort_values(ascending=False).plot.bar(figsize=(32, 6),title=f'{model_name} feature importance')
  plt.show()

  plt.figure(figsize=(30, 6))
  sns.lineplot(x=test.index, y=test['y'], label='Real')
  sns.lineplot(x=test.index, y=y_pred[model_name], label='Predicted')
  plt.title('Real vs Predicted Values')
  plt.legend()
  plt.show()